In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# packages computer assignment 1: 
import pandas as pd
from scipy.optimize import minimize
from scipy import stats
from pandas.tseries.offsets import DateOffset
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(suppress=True)
from scipy.stats import norm
import statsmodels.api as sm
import math
from scipy.optimize import root

np.random.seed(123)

# Part 2

### Previous assignment:

In [ ]:
def calc_euler(S0, sigma, r, delta_t,N):
    """
    goal: calculating the price of an asset using the Euler Method. 

    Input: 
    - S0 (float): Initial stock value (-1 value of the SP500 31 august)
    - Sigma (float): fitted volatility  
    - r (float): risk free rate
    - delta_t (float): T/N
    - N (float): Number of steps (Euler)

    Output: 
    - stock (array)
    """

    stock = np.zeros(N+1)
    var_normdistributed = np.random.normal(0, 1, size=N)
    stock[0] = S0

    for n in range(0,N):
        stock[n+1] = stock[n] + stock[n] * (r *delta_t + sigma * np.sqrt(delta_t) * var_normdistributed[n])
    return stock

def calc_geombrownian(S0, sigma, T):
    """
    Geometric Brownian motion

    Input: 
    - S0 (float): Initial stock value (-1 value of the SP500 31 august)
    - T (float): Maturity (0.25 = quarterly maturity) 
    - Sigma (float): fitted volatility 
    
    Output: 
    - ST (float): stock value 
    """

    var_normdistributed = np.random.normal(0, 1)    
    ST = S0 *np.exp((r - 0.5 * sigma**2)*T + sigma* np.sqrt(T)*var_normdistributed)
    
    return ST


def calc_ornsteinuhlenbeck():
    return


def calc_averagepayoff(S0, sigma, r, delta_t,K, T, N=3,M = 100, type = 'eu_call', method = 'euler'):
    """
    Calculate the average payoff using different methods
    
    Input: 
    - S0 (float): Initial stock value (-1 value of the SP500 31 august)
    - T (float): Maturity (0.25 = quarterly maturity) 
    - Sigma (float): fitted volatility  
    - r (float): risk free rate
    - delta_t (float): T/N
    - N (float): Number of steps (Euler)
    - M (float): number of simulations
    - type (string): eu_call or asian_call
    - method (string): euler or geom_brownian

    Output: 
    - option_price (float)
    """
    payoffs = np.zeros(M)

    if type == 'eu_call':
        for j in range(M):
            if method == 'euler': 
                stock_path = calc_euler(S0, sigma, r, delta_t, N=N) 
                payoffs[j] = np.maximum(stock_path[-1] - K, 0)       
            if method == 'geom_brownian':
                stock_path = calc_geombrownian(S0, sigma, T)
                payoffs[j] = np.maximum(stock_path - K, 0)

    if type == 'asian_call':
        for j in range(M):
            stock_path = calc_euler(S0,sigma, r, delta_t, N=N)
            payoffs[j] = np.maximum(np.mean(stock_path[-21:]) - K, 0)
            
    average_payoff = np.mean(payoffs)
    option_price = np.exp(-r * T) * average_payoff

    return option_price


    
def make_tree(S0, n, u, d):
    """
    make a tree
    
    input:
    - n (int) number of time periods
    - u (double) up probability
    - d (double) down probability
    """
    
    tree = np.zeros((n + 1, n + 1))

    for j in range(n + 1):
        for i in range(j + 1):
            tree[i, j] = S0 * (d ** i) * (u ** (j - i))
    return tree


    r = 0.0086
    n = 3  # Number of periods

    tree = make_tree(S0, n, u, d)



### Current assignment

In [3]:
sp500index_file = r'data\SP500.csv'
sp500index_df = pd.read_csv(sp500index_file)
sp500index_df.head(10)

sp500index_df['DATE'] = pd.to_datetime(sp500index_df['DATE'])
sp500index_df.set_index('DATE', inplace=True)

cutoff_date = pd.Timestamp('2023-08-31')
num_months= 10*(11+4) - 44

start_date = cutoff_date - DateOffset(months=num_months)
filtered_sp500index_df = sp500index_df.loc[start_date:cutoff_date]

filtered_sp500index_df = filtered_sp500index_df[
    filtered_sp500index_df.apply(lambda row: '.' not in row.values, axis=1)
].astype(float)


S0 = filtered_sp500index_df.iloc[-1][0]
logreturn = np.log(filtered_sp500index_df / filtered_sp500index_df.shift(-1)).dropna()

C:\Users\bramg\AppData\Local\Temp\ipykernel_33676\413326341.py:15: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  filtered_sp500index_df.apply(lambda row: '.' not in row.values, axis=1)


In [125]:
def fit_ols_logreturn(logreturn):
    """
    Fit OLS to calculate mu_tilde and sigma_tilde using statsmodels.

    input: 
    - data (dataframe): SP500 data

    output: 
    - mu_tilde (double)
    - sigma_tilde (double)
    """
    y = logreturn
    X = sm.add_constant(np.ones(len(y)))  # Adding a constant for intercept
    
    model = sm.OLS(y, X).fit()
    
    mu_tilde = model.params[0]
    sigma_tilde = np.sqrt(model.mse_resid)
    
    return mu_tilde, sigma_tilde


mu, sigma = fit_ols_logreturn(logreturn)

In [126]:
logreturn

,SP500
DATE,
2014-11-01,0.004197
2014-12-01,0.031533
2015-01-01,-0.053439
2015-02-01,0.017549
2015-03-01,-0.008485
...,...
2023-03-01,-0.014536
2023-04-01,-0.002479
2023-05-01,-0.062719


## Mean reverting: 

$$dX_{t}=\theta(\mu-X_{t})dt+\sigma dW_{t}$$

(4700.583104594036, -0.007571228704087814, 2.8063049790815144)

In [146]:
mu

-0.007571228704087814

In [5]:
def make_meanreverting_binmoialtree(S0, mu, alpha, sigma, r, dt, N):
    """
    make a tree

    """
    df = np.exp(-r * dt) #discount factor (differs over time?)

    u = np.exp(sigma * np.sqrt(dt))  # up factor
    d = 1 / u  # down factor
    p = 0.5 + 0.5 * (alpha * (mu - np.log(S0)) * np.sqrt(dt) / sigma)  # probability of up move

    price_tree = np.zeros((N+1, N+1))
    price_tree[0, 0] = S0

    for i in range(1, N + 1):
        price_tree[i, 0] = price_tree[i - 1, 0] * u
        for j in range(1, i + 1):
            price_tree[i, j] = price_tree[i - 1, j - 1] * d
    return price_tree, p, df

def value_meanreverting_binomialtree(price_tree, N, p, df, K):
    option_tree = np.maximum(price_tree[-1] - K, 0)  # Replace K with strike price
    print(p)
    # Backward induction for option pricing
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            option_tree[j] = (p * option_tree[j] + (1 - p) * option_tree[j + 1]) * df
    return option_tree[0]

T= 3 #steps
N = 3 #months
K = 4600 
dt = T/N
alpha = 0.2
r = 0.05

price_tree, p, df = make_meanreverting_binmoialtree(S0, mu, alpha, sigma, r, dt, N)
value_meanreverting_binomialtree(price_tree, N, p, df, K)

-18.18428374965183


-903447.4100293848

## Binomial 

## Mean Reverting using monte carlo: 


In [150]:
import numpy as np
import scipy.optimize as optimize

def log_likelihood(params, data):
    theta, mu, sigma = params
    dt = delta_t  # Assuming time increments are 1, adjust as needed
    term1 = -np.log(sigma * np.sqrt(2 * np.pi))
    term2 = -((data[1:] - data[:-1] - theta * (mu - data[:-1]) * dt) ** 2) / (2 * sigma ** 2 * dt)
    return np.sum(term1 + term2)

def fit_ou_parameters(log_returns, method):
    # Initial guess for the parameters
   # Initial guess for the parameters
    initial_guess = [0.5, np.mean(log_returns), np.std(log_returns)]

    # Minimize the negative log likelihood
    result = optimize.minimize(lambda params: -log_likelihood(params, log_returns),
                               initial_guess, method=method,
                               bounds=((1e-5, None), (None, None), (1e-5, None)))

    if result.success:
        theta, mu, sigma = result.x
        return theta, mu, sigma
    else:
        raise Exception('Optimization failed. Message: {}'.format(result.message))

# Example usage

theta, mu, sigma = fit_ou_parameters(logreturn.values, method='L-BFGS-B')

In [152]:
def sim_ornsteinuhlenbeck(S0, sigma, r, delta_t,  theta, N=N, mu=K):
    """
    Ornstein Ulhenbeck proce
    
    """    
    stock_path = np.zeros(N+1)
    stock_path[0] = S0

    for i in range(1, N+1):
        Z = np.random.normal(0, 1)
        stock_path[i] = stock_path[i-1] + theta * (mu - stock_path[i-1]) * delta_t + sigma * Z * np.sqrt(delta_t)

    return stock_path

sim_ornsteinuhlenbeck(S0, sigma, r, delta_t,  theta, N=N, mu=mu)

array([4517.01      , -791.13444558,  138.55480761, ...,    0.11197136,
         -0.07039425,    0.0259466 ])

In [151]:


def sim_geombrownian(S0, sigma, T, r):
    """
    Geometric Brownian motion

    Input: 
    - S0 (float): Initial stock value (-1 value of the SP500 31 august)
    - T (float): Maturity (0.25 = quarterly maturity) 
    - Sigma (float): fitted volatility 
    
    Output: 
    - ST (float): stock value 
    """

    var_normdistributed = np.random.normal(0, 1)    
    ST = S0 *np.exp((r - 0.5 * sigma**2)*T + sigma* np.sqrt(T)*var_normdistributed)
    
    return ST

In [117]:
def sim_ornstein_uhlenbeck_vectorized(S0, sigma, r, delta_t, theta, N, mu):
    Z = np.random.normal(0, 1, N)
    stock_path = np.zeros(N + 1)
    stock_path[0] = S0

    for i in range(1, N + 1):
        drift = theta * (mu - stock_path[i - 1]) * delta_t
        diffusion = sigma * Z[i - 1] * np.sqrt(delta_t)
        stock_path[i] = stock_path[i - 1] + drift + diffusion

    return stock_path
mu
# sim_ornstein_uhlenbeck_vectorized(S0, sigma, r, delta_t, theta, N, mu)

-0.007442677166456411

In [ ]:
def sim_ornsteinuhlenbeck_vectorized(S0, sigma, r, delta_t, theta, N, mu):
    Z = np.random.normal(0, 1, N)
    stock_path = S0 + np.cumsum(theta * (mu - S0) * delta_t + sigma * Z * np.sqrt(delta_t))
    np.insert(stock_path, 0, S0)
    return np.insert(stock_path, 0, S0)  # Insert initial value at the start

def sim_montecarlo(function, S0, sigma, r, delta_t, theta, N, mu):
    
    if method == 'ornstein_uhlenbeck': 
        stock_path = sim_ornsteinuhlenbeck_vectorized(S0, sigma, r, delta_t, theta, N, mu)            
    elif method == "geometric_brownian":
        stock_path = sim_geombrownian(S0, sigma, T, r)
    calc_payoff

def calc_payoff(option_type, stock_path, K):
    """"
    
    """

    if option_type =="eu_call":
        return np.maximum(stock_path - K, 0)
    
    if option_type =="eu_put":
        return np.maximum(K - stock_path, 0)
    else: 
        return ValueError('Invalid option type')    

def calc_averagepayoff():
    """
    """

    for j in range(M):


In [63]:


def sim_geombrownian(S0, sigma, T, r):
    """
    Geometric Brownian motion

    Input: 
    - S0 (float): Initial stock value (-1 value of the SP500 31 august)
    - T (float): Maturity (0.25 = quarterly maturity) 
    - Sigma (float): fitted volatility 
    
    Output: 
    - ST (float): stock value 
    """

    var_normdistributed = np.random.normal(0, 1)    
    ST = S0 *np.exp((r - 0.5 * sigma**2)*T + sigma* np.sqrt(T)*var_normdistributed)
    
    return ST



def run_montecarlosimulation(function, )
    for j in range(M):


def calc_averagepayoff(S0, sigma, r, delta_t,K, T, N=3,M = 100, type = 'eu_call', method = 'ornstein_uhlenbeck', theta = None):

    payoffs = np.zeros(M)

    for j in range(M):
        if method == 'ornstein_uhlenbeck': 
            stock_path = sim_ornsteinuhlenbeck(S0, sigma, r, delta_t, theta, N=N, mu=S0)[-1]            
        elif method == "geometric_brownian":
            stock_path = sim_geombrownian(S0, sigma, T, r)
            
        payoffs[j] = np.maximum(stock_path - K, 0)       

    average_payoff = np.mean(payoffs)
    option_price = np.exp(-r * T) * average_payoff

    return option_price, payoffs


T = 0.25
strike = 4600
N = 1000
delta_t = T / N
M = 1

methods = ['ornstein_uhlenbeck', 'geometric_brownian']
results = {}
stock_paths = {}
thetas = np.arange(0,1.2, 0.1)


for method in methods:
    if method=='ornstein_uhlenbeck':
        results[method] = {}
        stock_paths[method] = {}
        for theta in thetas:
            print(theta) 
            results[method][theta] , stock_paths[method][theta] = calc_averagepayoff(S0, sigma, r, delta_t,K, T,N , M = 1, type = 'eu_call', method = method, theta = theta)
    else: 
        results[method] , _ = calc_averagepayoff(S0, sigma, r, delta_t,K, T,N , M = 100, type = 'eu_call', method = method)

0.0
0.1
0.2
0.30000000000000004
0.4
0.5
0.6000000000000001
0.7000000000000001
0.8
0.9
1.0
1.1


In [53]:
stock_paths

{'ornstein_uhlenbeck': {0.0: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  0.1: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  0.2: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.